In [1]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
os.environ["TORCH_USE_CUDA_DSA"] = "1"

In [2]:
%%time
import lib.utils.utils as utils
from run import get_images
#import lib.config as config
from lib.model.base_model import BaseModel
#import lib.roi as roi
#import lib.split_pages as spages
from lib.utils.promptLoader import PromptLoader

CPU times: total: 562 ms
Wall time: 637 ms


c:\Users\ika11kg\OneDrive - The Royal Botanic Gardens, Kew\Documents\lightfootcat\LightfootCatalogue\lib\utils\text_utils.py:47: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if family is None or family is '':


In [3]:
image_path = "images/cropped/The_Lightfoot_Herbarium_05_cropped_2.jpg"
image_dir = "images/cropped/"
default_yaml = "./prompts/default_v2_fromtext.yaml"
conversation = None

In [4]:
print(image_path)

images/cropped/The_Lightfoot_Herbarium_05_cropped_2.jpg


In [5]:
images = sorted(utils.load_images(image_dir))

In [6]:
#model = BaseModel("qwen_model", prompt=default_yaml, conversation=conversation, max_new_tokens = 130000, batch_size=1, temperature=1, save_path="./outputs/lightfootcat/")

In [7]:
#outputs = model(images, save=False, debug=False, save_file_name="two_images")

In [8]:
from lib.utils.text_utils import convertToTextBlocks
#extracted_text = model.extract_text(images)
with open("./temp.txt", "r") as f_:
    extracted_text = f_.read()

text_blocks = convertToTextBlocks(extracted_text)

In [9]:
text_blocks

{'Dicotyledones': ['ACERACEAE\n\nAcer campestre L.\n\n1 folder. Acer campestre [TA]\n\nAcer pseudoplatanus L.\n\n2 folders.\n\nFolder 1. Acer Pseudo-Platanus [G]. i. "Maple. Bulls: [Bulstrode] Park" [JL]\n\nFolder 2. Acer Pseudo-Platanus [TA].',
  'AMARANTHACEAE\n\nAmaranthus lividus L., Flora Europaea 1: 110 (1964)\n\n1 folder. Amaranthus Blitum [TA]. i. Cites Ray\'s Syn. 1957. ii. "Blite Amaranth. Aug.: It is often found on Dunghills in the neighbourhood of London. I gather\'d this on a Dunghill at Fulham near London" [JL]. iii. "Amaranthus Blitum. Monoec: 5. and:" [JL]',
  'ARALIACEAE\n\nHedera helix L.\n\n1 folder. Hedera Helix [TA].',
  'ARISTOLOCHIACEAE\n\nAsarum europaeum L.\n\n1 folder. Asarum europaeum [TA]. i. "Asarum europaeum Lin: Asara-bacca. Anglis. ex horto" [JL].',
  'BALSAMINACEAE\n\nImpatiens noli-tangere L.\n\n1 folder. Impatiens Noli me tangere [G].',
  'BETULACEAE\n\nAlnus glutinosa (L.) Gaertn.\n\n1 folder. Betula Alnus [TA].\n\nBetula nana L.\n\n1 folder. Betula 

In [10]:
a = 'Folder 3. Euphorbia platyphylllos [G]. i. "Euphorbia platyphylllos? ex horto Kewensi, given for Euphorbia verrucosa, but I think falsely" [JL]. ii. "Euphorbia verrucosa is at least a Biennial if not a Perennial. According to Morrison, Lin: & Gerard. Gerard says the Stalks aredecumbent. Euphorb. platyphylllos is an annual according to all Authors, & always erect. The Leaves being lightly villose is not enough to determine the Species, since Bauhine\'s Pinax says the Leaves of the latter are so. mine from Harfield has a few stragling Hares upon it, but that in Kew Garden is more hairy, which is the only difference I am able to find. Amanitat: Acad: says the Involucella of E. verrucosa are integerrima. And in Species Plant: he says of the same Fructus verucosus simulg: villosus" [JL]. Folder 4. Euphorbia platyphylllos [G]. i. "Euphorbia verrucosa? seu potius platyphylllos Lin: In a Corn field at the Bottom of Oak of Honour Hill, half a mile from Peckham, by Mr. Curtis" [JL]. Folder 5. Euphorbia platyphylllos [G]. i. "Euphorbia verrucosa? s. potius platyphylllos Lin: In Corn-Fields near Wakefield, Yorkshire" [JL]. Folder 6. Euphorbia platyphylllos [TA]. i. "Euphorbia platyphylllos Lin: From Battles Well near Harefield, Middlesex" [JL]. ii. "compar\'d with the specimens of Linnaeus w. one of w. it agrees, but not I think w. his first & principal, w. has much larger & wider Leaves than this" [JL] [apparently a note added to 6,i at a later date]. Folder 7. Euphorbia stricta Linn.? and E. verrucosa Huds.? [G]. i. "Euphorbia verucosa? seu potius platyphylllos. In a Corn field at the Foot of Glastonbury Tor" [JL]. Folder 8. Euphorbia verrucosa [TA]; E. platyphylllos [G]. i. "Given for Euphorbia verrucosa. Lin: An non potius Euphorbia platyphylllos? Lin: Collected at Selburn in Hants by Mr. Yalden" [JL]. Euphorbia portlandica L. 1 folder and 3 sheets. Folder 1. Euphorbia portlandica [TA]. i. "Exmouth in Devonshire. Euphorbia portlandica Lin:" [JL]. Sheet 1. Euphorbia portlandica [G]. i. "From the Isle of Caldy. Pembroke" [TA]. Sheet 2. Euphorbia portlandica [G]. i. "Ex Brounton [probably Braunton]. Devonshire" [TA]. Sheet 3. Euphorbia portlandica [G]. Mercurialis annua L. 3 folders. Folder 1. Mercurialis annua. [G]. i. "Dorchester" [JL]. Folder 2. Mercurialis annua [G]. i. "Dorsetshire" [JL]. Folder 3. Mercurialis annua [TA]. FRANKENIACEAE Frankenia laevis L. 2 folders. Folder 1. Frankenia laevis [TA]. i. "Frankenia laevis. At Cley, on the Beach towards Salthouse [small villages on the north Norfolk coast near Blakeney]" [B]. Folder 2. Frankenia laevis [TA]. Frankenia pulverulenta L., Flora Europaea 2: 295 (1968) 1 sheet. Frankenia pulverulenta Lin: [JL]. i. "Frankenia pulverulenta. Africa leg. Thouars" [JL]; "Ammania R.B." [? Robert Brown; in pencil and refers to the plant at the top of the sheet; the other two plants are Peplis portula L. (Lythraceae)]. ii. "23. Alsine maritima superna foliis Chamaesyces. I.R.H. Franca maritima, quadrifolia annua, supina, chamaesyces folio & facie, flore ex albo purpurascente. Mich: Nov. Gen. 23. Frankenia pulverulenta. Flos in quinque petala dividitur ad tubulum cohaerentia, basi denuo petala separatum & arcte amplex... [illegible] fructum oblongum pentagonum, monangium, plurimus seminibus fatum. Calyx longus striatus, quinquefidus est. Flores arcte geniculis ramulosum adnascuntum" [JL]. GENTIANACEAE Blackstonia perfoliata (L.) Huds. 1 folder. Chlora perfoliata [G],Flora Scotica ed. 1, 200, 1139 (1777); Chironia centaureum [TA]. i. "Chlora perfoliata. Lin: Syst: Nat: Blackstonia perfoliata. Huds." [JL]; "Chironia centaureum Curtis and properly so. T.A." [TA] [later deleted by Goodenough]. [See Lightfoot\'s comment p. 1139 of Flora Scotica ed. 1: "We suspect some deception in this plant being term\'d a native of Scotland, as we never found it." It has been recorded from Kirkcudbright, however, according to C. T. & W., p. 644.]\n\nCentaurium erythraea Rafn 1 folder. Chironia Centaureum [TA]; Eng. Botany Sowerby [G]; Gentiana centaurium Huds. [G]. [Nine specimens of C. erythraea, and one of C. pulchellum at bottom left corner of left side of folder, labelled "1" in pencil by JKB.]\n\nCentaurium pulchellum (Sw.) Druce [See under C. erythraea]\n\nCicendia filiformis (L.) Delarbre 1 folder. Gentiana filiformis [TA]; Exacum filiforme. English Botany. Sowerby [G]. i. "Pool [Poole, Dorset] Heath & Cornwall" [JL]. ii. "Isle of Purbeck near Corf [Corfe, Dorset]" [JL]. iii. "Isle of Purbeck. Dorset" [JL]. iv. "The little yellow Marsh Centory. In Rotten marshy Gound in Dorsetshire, Devonshire, & Cornwall. July & August" [JL].\n\nGentiana pneumonanthe L. 1 folder. Gentiana Pneumonanthe [TA]. i. "Gentiana Pneumonanthe. Lin: Stockton Heath near York" [JL].\n\nGentiana amarella (L.) Börner 1 folder. Gentiana Amarella [TA], Flora Scotica ed. 1, 152 (1777).\n\nGentianella campestris (L.) Börner 2 folders. Folder 1. Gentiana campestris vide Pentandria Digynia [TA], Flora Scotica ed. 1, 152 (1777). i. "Gen: campestris. Teesdale" [JL]. ii. "Gentiana campestris Hud:" [? Banks]. iii. "Gentiana campestris. Wales. Mr Banks" [JL].\n\nFolder 2. Gentiana amarella [G]. i. "Scotland" [JL].\n\nSwertia perennis L., Flora Europaea 3: 67 (1972) 1 folder. Swertia perennis [TA]. i. "Swertia perennis Lin: ex Helvetia a D. Banks" [JL]. [One of Banks\' specimens from Switzerland.] [For distribution see Flora Europaea: "Wet places, mainly in the mountains. From NC France & NW Russia southwards to N Spain, Bulgaria & S Ural."]'
b = re.compile("\n\n(?=[A-Z ]+\n|.+?[aA][cC][eE][aA][eE])")

NameError: name 're' is not defined

In [14]:
re.match(b, a)

In [10]:
import re

def decor(text):
    print("="*10)
    print(text)
    print("="*10)

In [11]:
def split_into_smaller_chunks(large_block, max_chunk_size=3000):

    # Split the large block into family name at the start and the rest
    family_name, text_block = list(
                                filter(None, 
                                       re.split(r"^([A-Z]+)", large_block)
                                       )
                                )
    
    # Define static small chunks
    small_chunks = [text_block[i:min(i+max_chunk_size, len(text_block))] for i in range(0, len(text_block), max_chunk_size)]

    # Setting cut_off index for next chunk to add to final chunks
    final_chunks = []
    cut_off = 0

    # Iterate through all the static chunks
    for ind, small_chunk in enumerate(small_chunks):
        # Define the regex for the species name    
        species_name = re.compile("^[A-Z]+ [a-z]+ \(?[A-Z]+\.\)?", re.IGNORECASE)

        # Split the small chunk by line and then remove the lines that were added to the previous chunk using cut_off value
        small_chunk_splitted = small_chunk.split("\n")
        small_chunk_splitted = small_chunk_splitted[cut_off:]
        small_chunk = "\n".join(small_chunk_splitted)
        
        # Define the next chunk
        next_chunk =  None if ind+1 >= len(small_chunks) else small_chunks[ind+1]
        
        lines_to_add = []

        # If at end of list
        if next_chunk is None:
            # If the size of the small chunk is smaller than 5% of the max chunk size then just add it to the previous chunk
            if len(small_chunk) <= (max_chunk_size * 0.05):
                final_chunks[-1] += "\n" + small_chunk
            # If not add it to final chunks
            else:
                small_chunk = family_name + "\n\n" + small_chunk
                final_chunks.append(small_chunk)
            break
        else:
            # Split the next chunk by line
            next_chunk_splitted = next_chunk.split("\n")

            # For each line in next chunk
            for ind, line in enumerate(next_chunk_splitted):
                # check if the line matches the species name pattern
                if re.match(species_name, line) is None:
                    # if so add the line to the lines_to_add
                    lines_to_add.append(line)
                else:
                    # if not break the loop and update cut_off
                    cut_off = ind
                    break
        
        # Join all the lines together and append it to the small chunk
        small_chunk += "\n".join(lines_to_add)
        # Add the family name back
        small_chunk = family_name + "\n\n" + small_chunk
        # Add it to final chunks
        final_chunks.append(small_chunk)
    

    return final_chunks

                

In [ ]:
small_chunks = split_into_smaller_chunks(sample_block)

NameError: name 'sample_block' is not defined

In [133]:
for i in small_chunks:
    decor(i)

COMPOSITAE



Anaphalis margaritacea (L.) Benth. 2 folders.
Folder 1. Gnaphalium margaritaceum [G]. i. "Gnaph: armeria s:d to

be British. Bulls: [Bulstrode Park]" [JL].
Folder 2. Gnaphalium margaritaceum [TA].

Antennaria dioica (L.) Gaertn. 3 folders.
Folder 1. Gnaphalium dioicum [TA]. i. "Newfoundland [A. neodioica Greene] Mr. Banks" [JL]. ii. "Gnaphalium dioicum on Stratton Heath ab' 7 miles from Norwich. Mr. John Pitchford" [B]. Folder 2. Gnaphalium dioicum [G]. i. "Gnaph: dioic: mas & foem. one Specimen was 15 Inches high. a var: only" [JL]. Folder 3. Gnaphalium dioicum [TA].

Antennaria neodioica Greene [See under A. dioica]

Anthemis arvensis L. 1 folder. Anthemis arvensis [TA].

Anthemis cotula L. 1 folder. Anthemis cotula [TA]. i. "Anthemis cotula" [JL]. ii. "This I take to be the Anthemis arvensis by the Road Side going to Maidenhead" [JL].

Arnosera minima (L.) Schweigg. & Koerte 1 folder. Hyoseris minima [TA], Flora Scotica ed. 1, 442 (1777). i. "Hyoseris minima, near Lang